In [100]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import warnings
warnings.filterwarnings("ignore")
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r
answers = {}

In [101]:
### Rating baseline: compute averages for each user, or return the global average if we've never seen the user before

allRatings = []
userRatings = defaultdict(list)

for user,book,r in readCSV("train_Interactions.csv.gz"):
    r = int(r)
    allRatings.append(r)
    userRatings[user].append(r)

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
for u in userRatings:
    userAverage[u] = sum(userRatings[u]) / len(userRatings[u])

predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.csv"):
    if l.startswith("userID"):
    #header
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    if u in userAverage:
        predictions.write(u + ',' + b + ',' + str(userAverage[u]) + '\n')
    else:
        predictions.write(u + ',' + b + ',' + str(globalAverage) + '\n')

predictions.close()

### Would-read baseline: just rank which books are popular and which are not, and return '1' if a book is among the top-ranked

bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/2: break

predictions = open("predictions_Read.csv", 'w')
for l in open("pairs_Read.csv"):
    if l.startswith("userID"):
    #header
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    if b in return1:
        predictions.write(u + ',' + b + ",1\n")
    else:
        predictions.write(u + ',' + b + ",0\n")

predictions.close()

In [102]:
#Load Data from Raw File
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
booksPerUser = defaultdict(set)
usersPerBook = defaultdict(set)
booksInTrain=set()
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))
    booksPerUser[u].add(b)
    usersPerBook[b].add(u)
    booksInTrain.add(b)

In [103]:
### Would Read Prediction---------------------------------------------------------------------------
### ------------------------------------------------------------------------------------------------
# JaccardSimilarity Func
def JaccardSimilarity(set1,set2):
    inter=len(set1.intersection(set2))
    uni=len(set1.union(set2))
    return inter/uni

# define newreturn
newreturn1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    newreturn1.add(i)
    if count > 0.72*totalRead: break
        
#threshold
pct=0.02631578947368421### if use 'and': 0.001072961373390558  #
import csv
with open('predictions_Read.csv', 'w',newline="") as f:
    writer = csv.writer(f)
    for l in open("pairs_Read.csv"):
        if l.startswith("userID"):
            line=['userID','bookID','prediction']
            writer.writerow(line)
            continue
        u,b = l.strip().split(',')
        books=booksPerUser[u]
        users=usersPerBook[b]
        sim_max=0
        for comparedBook in books:
            sim_ij=JaccardSimilarity(users,usersPerBook[comparedBook])
            if sim_ij>sim_max:
                sim_max=sim_ij
        threshold=pct
        if sim_max>threshold or b in newreturn1:
            predict=1
        else:
            predict=0
        line=[u,b,str(predict)]
        writer.writerow(line)

In [117]:
betaU = {}
betaI = {}
for u in ratingsPerUser:
    betaU[u] = 0

for b in ratingsPerItem:
    betaI[b] = 0

# MSE Function--------------------------------------------------------
def MSE(pred, label):
    err_sqr = [(x-y)**2 for x,y in zip(pred,label)]
    return sum(err_sqr) / len(err_sqr)

ratingMean =  sum([r for (_,_,r) in ratingsTrain]) / len(ratingsTrain)
alpha = ratingMean
def iterate(lamb):
    newAlpha = 0
    for u,b,r in ratingsTrain:
        newAlpha += r - (betaU[u] + betaI[b])
    alpha = newAlpha / len(ratingsTrain)
    for u in ratingsPerUser:
        newBetaU = 0
        for b,r in ratingsPerUser[u]:
            newBetaU += r - (alpha + betaI[b])
        betaU[u] = newBetaU / (lamb + len(ratingsPerUser[u]))
    for b in ratingsPerItem:
        newBetaI = 0
        for u,r in ratingsPerItem[b]:
            newBetaI += r - (alpha + betaU[u])
        betaI[b] = newBetaI / (lamb + len(ratingsPerItem[b]))
    mse = 0
    for u,b,r in ratingsTrain:
        prediction = alpha + betaU[u] + betaI[b]
        mse += (r - prediction)**2
    regularizer = 0
    for u in betaU:
        regularizer += betaU[u]**2
    for b in betaI:
        regularizer += betaI[b]**2
    mse /= len(ratingsTrain)
    return mse, mse + lamb*regularizer
lamb=4
mse,objective = iterate(lamb)
newMSE,newObjective = iterate(lamb)
iterations = 2

In [118]:
while iterations < 200 or objective - newObjective > 0.0001:
    mse, objective = newMSE, newObjective
    newMSE, newObjective = iterate(lamb)
    iterations += 1
    print("Objective after "
        + str(iterations) + " iterations = " + str(newObjective))
    print("MSE after "
        + str(iterations) + " iterations = " + str(newMSE))

Objective after 3 iterations = 25012.763605953598
MSE after 3 iterations = 1.132855175417788
Objective after 4 iterations = 24985.006216807353
MSE after 4 iterations = 1.1327414062485137
Objective after 5 iterations = 24960.103593520373
MSE after 5 iterations = 1.1326605592491956
Objective after 6 iterations = 24939.576356882426
MSE after 6 iterations = 1.132593576922832
Objective after 7 iterations = 24922.986746012637
MSE after 7 iterations = 1.1325367268987514
Objective after 8 iterations = 24909.69036613869
MSE after 8 iterations = 1.1324882631017639
Objective after 9 iterations = 24899.098666199163
MSE after 9 iterations = 1.1324468814891475
Objective after 10 iterations = 24890.710573392134
MSE after 10 iterations = 1.132411494895979
Objective after 11 iterations = 24884.108251974205
MSE after 11 iterations = 1.1323811828762882
Objective after 12 iterations = 24878.946786363886
MSE after 12 iterations = 1.1323551669565615
Objective after 13 iterations = 24874.943320424456
MSE aft

Objective after 90 iterations = 24868.040626729806
MSE after 90 iterations = 1.13217623852693
Objective after 91 iterations = 24868.041416096974
MSE after 91 iterations = 1.1321762343668265
Objective after 92 iterations = 24868.042129119003
MSE after 92 iterations = 1.1321762306095757
Objective after 93 iterations = 24868.04277317262
MSE after 93 iterations = 1.1321762272161209
Objective after 94 iterations = 24868.043354925496
MSE after 94 iterations = 1.1321762241512616
Objective after 95 iterations = 24868.043880399407
MSE after 95 iterations = 1.1321762213831563
Objective after 96 iterations = 24868.04435503523
MSE after 96 iterations = 1.1321762188830458
Objective after 97 iterations = 24868.044783749225
MSE after 97 iterations = 1.1321762166250282
Objective after 98 iterations = 24868.045170981917
MSE after 98 iterations = 1.132176214585643
Objective after 99 iterations = 24868.045520745443
MSE after 99 iterations = 1.1321762127436763
Objective after 100 iterations = 24868.045836

Objective after 175 iterations = 24868.048783397007
MSE after 175 iterations = 1.1321761955669527
Objective after 176 iterations = 24868.048783534618
MSE after 176 iterations = 1.1321761955662166
Objective after 177 iterations = 24868.048783659273
MSE after 177 iterations = 1.1321761955655572
Objective after 178 iterations = 24868.048783771137
MSE after 178 iterations = 1.1321761955649727
Objective after 179 iterations = 24868.048783872982
MSE after 179 iterations = 1.1321761955644563
Objective after 180 iterations = 24868.048783964772
MSE after 180 iterations = 1.1321761955639449
Objective after 181 iterations = 24868.04878404747
MSE after 181 iterations = 1.13217619556353
Objective after 182 iterations = 24868.04878412211
MSE after 182 iterations = 1.1321761955631475
Objective after 183 iterations = 24868.048784189916
MSE after 183 iterations = 1.1321761955627843
Objective after 184 iterations = 24868.04878425097
MSE after 184 iterations = 1.132176195562439
Objective after 185 iterat

In [119]:
print("On ValidationSet")
specialCase=0
predValid=[]
alwaysPredictMean = [ratingMean for _ in ratingsValid]
labels = [r for _,_,r in ratingsValid]
specialCase=0
for (u,b,_) in ratingsValid:
    if u not in ratingsPerUser or b not in ratingsPerItem:
        predValid.append(ratingMean)
        specialCase+=1
    else:
        prediction=alpha + betaU[u] + betaI[b]
        predValid.append(prediction)
mseValid=MSE(predValid,labels)
print("MSE on ValidationSet:{}".format(mseValid))
print("Always predict Mean:{}".format(MSE(alwaysPredictMean, labels)))
print("Special Case Number:{}".format(specialCase))

On ValidationSet
MSE on ValidationSet:1.4012896868699762
Always predict Mean:1.6802113179223874
Special Case Number:33


In [120]:
import csv
case=0
with open('predictions_Rating.csv', 'w',newline="") as f:
    writer = csv.writer(f)
    for l in open("pairs_Rating.csv"):
        if l.startswith("userID"):
            line=['userID','bookID','prediction']
            writer.writerow(line)
            continue
        u,i = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
        if u not in ratingsPerUser or i not in ratingsPerItem:
            predict=ratingMean
            case+=1
        else:
            predict=alpha + betaU[u] + betaI[i]
        line=[u,i,str(predict)]
        writer.writerow(line)
    
print("Special Case:{}".format(case))

Special Case:33


In [116]:
### Rating Prediction-------------------------------------------------------------------------------
### ------------------------------------------------------------------------------------------------
# reviewsPerUser & reviewsPerItem
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
for (u,i,r) in ratingsTrain:
    reviewsPerUser[u].append((i,r))
    reviewsPerItem[i].append((u,r))

N = len(ratingsTrain)
nUsers = len(reviewsPerUser)
nItems = len(reviewsPerItem)
users = list(reviewsPerUser.keys())
items = list(reviewsPerItem.keys())

ratingMean =  sum([r for (_,_,r) in ratingsTrain]) / len(ratingsTrain)
alpha = ratingMean

userBiases = defaultdict(float)
itemBiases = defaultdict(float)

# prediction Function-------------------------------------------------
def inner(x, y):
    return sum([a*b for a,b in zip(x,y)])
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item] +\
                    inner(userGamma[user], itemGamma[item])

# MSE Function--------------------------------------------------------
def MSE(pred, label):
    err_sqr = [(x-y)**2 for x,y in zip(pred,label)]
    return sum(err_sqr) / len(err_sqr)
# Gamma --------------------------------------------------------------
userGamma = {}
itemGamma = {}
K = 5
for u in reviewsPerUser:
    userGamma[u] = [random.random() * 0.1 - 0.05 for k in range(K)]
for i in reviewsPerItem:
    itemGamma[i] = [random.random() * 0.1 - 0.05 for k in range(K)]
# unpack -------------------------------------------------------------
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    global userGamma
    global itemGamma
    index = 0
    alpha = theta[index]
    index += 1
    userBiases = dict(zip(users, theta[index:index+nUsers]))
    index += nUsers
    itemBiases = dict(zip(items, theta[index:index+nItems]))
    index += nItems
    for u in users:
        userGamma[u] = theta[index:index+K]
        index += K
    for i in items:
        itemGamma[i] = theta[index:index+K]
        index += K
# cost func ----------------------------------------------------------
count=0
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(u,i) for (u,i,r) in ratingsTrain]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    #-----------------------------------------------------------------
    global count
    count+=1
    if cost<1.2 and count%4==0:
        predValid=[]
        labels = [r for _,_,r in ratingsValid]
        for (u,i,r) in ratingsValid:
            if u not in userBiases or i not in itemBiases:
                predValid.append(ratingMean)
            else:
                predValid.append(prediction(u,i))
        mseValid=MSE(predValid,labels)
        print("MSE on ValidationSet:{}".format(mseValid))
    #-----------------------------------------------------------------
    for u in users:
        cost += lamb*userBiases[u]**2
        for k in range(K):
            cost += lamb*userGamma[u][k]**2
    for i in items:
        cost += lamb*itemBiases[i]**2
        for k in range(K):
            cost += lamb*itemGamma[i][k]**2
    return cost

# derivative func ----------------------------------------------------
def derivative(theta, labels, lamb):
    unpack(theta)
    global N #len(trainData)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    dUserGamma = {}
    dItemGamma = {}
    for u in reviewsPerUser:
        dUserGamma[u] = [0.0 for k in range(K)]
    for i in reviewsPerItem:
        dItemGamma[i] = [0.0 for k in range(K)]
    for u,i,r in ratingsTrain:
        pred = prediction(u, i)
        diff = pred - r
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
        for k in range(K):
            dUserGamma[u][k] += 2/N*itemGamma[i][k]*diff
            dItemGamma[i][k] += 2/N*userGamma[u][k]*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
        for k in range(K):
            dUserGamma[u][k] += 2*lamb*userGamma[u][k]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
        for k in range(K):
            dItemGamma[i][k] += 2*lamb*itemGamma[i][k]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    for u in users:
        dtheta += dUserGamma[u]
    for i in items:
        dtheta += dItemGamma[i]
    return numpy.array(dtheta)

print("On TrainingSet:")
labels = [r for (_,_,r) in ratingsTrain]
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + # Initialize alpha
                                   [0.0]*(nUsers+nItems) + # Initialize beta
                                   [random.random() * 1 - 0.5 for k in range(K*(nUsers+nItems))], # Gamma
                             derivative, args = (labels, 10))
alwaysPredictMean = [ratingMean for _ in ratingsTrain]
labels = [r for _,_,r in ratingsTrain]
print("Always predict Mean:{}".format(MSE(alwaysPredictMean, labels)))

On TrainingSet:
MSE = 1.7756711205672684
MSE = 1.774538676671522
MSE = 1.770287595989867
MSE = 1.75723343818561
MSE = 1.7415472437173627
MSE = 1.741547249120045
MSE = 1.7415472490500343
Always predict Mean:1.741577477865528


In [109]:
print("On ValidationSet")
specialCase=0
predValid=[]
alwaysPredictMean = [ratingMean for _ in ratingsValid]
labels = [r for _,_,r in ratingsValid]
specialCase=0
for (u,i,r) in ratingsValid:
    if u not in userBiases or i not in itemBiases:
        predValid.append(ratingMean)
        specialCase+=1
    else:
        predValid.append(prediction(u,i))
mseValid=MSE(predValid,labels)
print("MSE on ValidationSet:{}".format(mseValid))
print("Always predict Mean:{}".format(MSE(alwaysPredictMean, labels)))
print("Special Case Number:{}".format(specialCase))

On ValidationSet
MSE on ValidationSet:1.5217824563229891
Always predict Mean:1.6802113179223874
Special Case Number:33


In [24]:
import csv
case=0
with open('predictions_Rating.csv', 'w',newline="") as f:
    writer = csv.writer(f)
    for l in open("pairs_Rating.csv"):
        if l.startswith("userID"):
            line=['userID','bookID','prediction']
            writer.writerow(line)
            continue
        u,i = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
        if u not in userBiases or i not in itemBiases:
            predict=ratingMean
            case+=1
        else:
            predict=prediction(u,i)
        line=[u,i,str(predict)]
        writer.writerow(line)
    
print("Special Case:{}".format(case))

Special Case:33


In [97]:
#Tensorflow Method
K=7
Lambda =1e-5
nsample=190000
Nepoch = 600
lr=0.001
opt="Adam"

import tensorflow as tf
userIDs,itemIDs = {},{}
for (u,i,_) in ratingsTrain:
#    u,i = d['user_id'],d['book_id']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)

if opt=="Adam":
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=lr,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name='Adam',
    )
elif opt=="SGD": 
    optimizer = tf.keras.optimizers.SGD(
        learning_rate=lr,
        momentum=0.0,
        nesterov=False,
        name='SGD',
    )

class LatentFactorModel(tf.keras.Model):
    def __init__(self, mu, K, lamb):
        super(LatentFactorModel, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.001))
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        self.lamb = lamb

    # Prediction for a single instance (useful for evaluation)
    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i] +\
            tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.betaU**2) +\
                            tf.reduce_sum(self.betaI**2) +\
                            tf.reduce_sum(self.gammaU**2) +\
                            tf.reduce_sum(self.gammaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        pred = self.alpha + beta_u + beta_i +\
               tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)

mu = sum([r for _,_,r in ratingsTrain]) / len(ratingsTrain)
modelLFM = LatentFactorModel(mu, K, Lambda)

def trainingStep(model, interactions):
    global nsample
    Nsamples = nsample
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactions)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleR.append(r)

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

lastObj = 1.0
for i in range(Nepoch):
    obj = trainingStep(modelLFM, ratingsTrain)
#     if abs(obj-lastObj)<1e-4:
#         break
    lastObj = obj
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 0.860635
iteration 20, objective = 0.85276055
iteration 30, objective = 0.8476717
iteration 40, objective = 0.843766
iteration 50, objective = 0.83557403
iteration 60, objective = 0.82694
iteration 70, objective = 0.8254044
iteration 80, objective = 0.8168975
iteration 90, objective = 0.8128576
iteration 100, objective = 0.80992866
iteration 110, objective = 0.7985431
iteration 120, objective = 0.7954058
iteration 130, objective = 0.7839033
iteration 140, objective = 0.78416497
iteration 150, objective = 0.7651531
iteration 160, objective = 0.7609721
iteration 170, objective = 0.7494892
iteration 180, objective = 0.7417577
iteration 190, objective = 0.7271344
iteration 200, objective = 0.7254221
iteration 210, objective = 0.7161714
iteration 220, objective = 0.70414793
iteration 230, objective = 0.6949621
iteration 240, objective = 0.6889049
iteration 250, objective = 0.6750182
iteration 260, objective = 0.6664377
iteration 270, objective = 0.660381
iteration 

In [98]:
# MSE Function--------------------------------------------------------
def MSE(pred, label):
    err_sqr = [(x-y)**2 for x,y in zip(pred,label)]
    return sum(err_sqr) / len(err_sqr)

alwaysPredictMean = [mu for _ in ratingsValid]
labels = [r for _,_,r in ratingsValid]
print("Always predict Mean:{}".format(MSE(alwaysPredictMean, labels)))
LFMPredictions=[]
case=0
for u,i,_ in ratingsValid:
    if u not in userIDs or i not in itemIDs:
        LFMPredictions.append(mu)
        case+=1
    else:
        LFMPredictions.append(modelLFM.predict(userIDs[u],itemIDs[i]).numpy())

validMSE=MSE(LFMPredictions, labels)
print(validMSE)
print("SpecialCase:{}".format(case))

Always predict Mean:1.6802113179223874
1.5731775331646407
SpecialCase:33


In [96]:
case=0
userIDs,itemIDs = {},{}
for (u,i,_) in ratingsTrain:
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
import csv
with open('predictions_Rating.csv', 'w',newline="") as f:
    writer = csv.writer(f)
    for l in open("pairs_Rating.csv"):
        if l.startswith("userID"):
            line=['userID','bookID','prediction']
            writer.writerow(line)
            continue
        u,i = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
        if u not in userIDs or i not in itemIDs:
            predict=mu
            case+=1
        else:
            predict=modelLFM.predict(userIDs[u],itemIDs[i]).numpy()
        line=[u,i,str(predict)]
        writer.writerow(line)
    
print("Special Case:{}".format(case))

Special Case:33


In [10]:
betaU_lst=modelLFM.betaU.numpy()
maxBeta,minBeta=0,0
for i,j in enumerate(betaU_lst):
    if j>maxBeta:
        maxBeta=j
        maxUser=i
    if j<minBeta:
        minBeta=j
        minUser=i
print([maxUser, minUser, maxBeta, minBeta])
userIDs,itemIDs = {},{}
for (u,i,_) in ratingsTrain:
#     u,i = d['user_id'],d['book_id']
    if not u in userIDs: 
        userIDs[u] = len(userIDs)
        if len(userIDs)==maxUser:
            maxUserNew=u
        if len(userIDs)==minUser:
            minUserNew=u
print([maxUserNew, minUserNew, maxBeta, minBeta])

[4570, 2533, 0.09651969, -0.09706633]
['u24801398', 'u12798255', 0.09651969, -0.09706633]
